# Anitápolis TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

/home/leo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
/home/leo/anaconda2/lib/python2.7/site-packages/fatiando/vis/myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


### Auxiliary functions

In [2]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [3]:
mypath = 'eqlayer_rs/direction_inc-37_dec-18_h300m/eqlayer_depth300m.pickle'

In [4]:
with open(mypath) as w:
        di = pickle.load(w)

In [5]:
xp = di['x']
yp = di['y']
zp = di['z']
dobs = di['upward']

In [6]:
xp

array([6915004.1884817, 6915004.1884817, 6915004.1884817, ...,
       6929876.0732984, 6929876.0732984, 6929876.0732984])

### Parameters of the initial model

In [7]:
M = 30 # number of vertices per prism
L = 14 # number of prisms
P = L*(M+2) + 1 # number of parameters

#figura
incs = -37.05
decs = -18.17
int_min = .2
int_max = 2.
intensity = np.linspace(int_min, int_max, 4)

# depth to the top, thickness and radius
z0_min = 50.
z0_max = 200.
z0 = np.linspace(z0_min, z0_max, 4)
dz = 300.
r = 2000.

x0 = 6921250.
y0 = 688000.

# main field
inc, dec = [-37.05, -18.17]

### Limits

In [8]:
# limits for parameters in meters
rmin = 100.
rmax = 3000.
y0min = 684931.
y0max = 695931.
x0min = 6918250.
x0max = 6926250.
dzmin = 10.
dzmax = 800.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [9]:
# variation for derivatives
deltax = 0.01*np.max(1000.)
deltay = 0.01*np.max(1000.)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [10]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [11]:
mypath = 'tfa_inversion/multiple_int_min%d_int_max%d_z0_min%d_z0_max%d' % (int_min, int_max, z0_min, z0_max)
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [12]:
# output of inversion
inversion = dict()

### Regularization parameters

In [13]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 0.01
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.00007  # adjacent radial distances within each prism
a2 = 0.0001   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.008     # vertically adjacent origins
a6 = 0.0000001   # zero order Tikhonov on adjacent radial distances
a7 = 0.0005     # zero order Tikhonov on thickness of each prism

In [14]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [15]:
itmax = 30
itmax_marq = 12
tol = 1.0e-4     # stop criterion

In [16]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [17]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
#inversion['initial_estimate'] = model0
#inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]

### Inversion

In [18]:
inversion_results = []
for z in z0:
    for i in intensity:
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([d_fit, m_est, model_est, phi_list, model_list, dobs - d_fit, res_list])

it:  0   it_marq:  0   lambda: 1e-02   misfit: 5.16415e+03
it:  0   it_marq:  1   lambda: 1e-01   misfit: 5.14009e+03
it:  0   it_marq:  2   lambda: 1e+00   misfit: 5.12242e+03
it:  0   it_marq:  3   lambda: 1e+01   misfit: 4.34829e+03
it:  0   it_marq:  4   lambda: 1e+02   misfit: 3.63286e+03
it:  1   it_marq:  0   lambda: 1e+01   misfit: 4.48622e+03
it:  1   it_marq:  1   lambda: 1e+02   misfit: 3.83147e+03
it:  1   it_marq:  2   lambda: 1e+03   misfit: 3.65205e+03
it:  1   it_marq:  3   lambda: 1e+04   misfit: 3.64513e+03
it:  1   it_marq:  4   lambda: 1e+05   misfit: 3.64018e+03
it:  1   it_marq:  5   lambda: 1e+06   misfit: 3.63482e+03
it:  1   it_marq:  6   lambda: 1e+07   misfit: 3.63285e+03
it:  0   it_marq:  0   lambda: 1e-02   misfit: 3.08551e+03
it:  0   it_marq:  1   lambda: 1e-01   misfit: 2.94914e+03
it:  0   it_marq:  2   lambda: 1e+00   misfit: 2.08477e+03
it:  0   it_marq:  3   lambda: 1e+01   misfit: 1.27334e+03
it:  1   it_marq:  0   lambda: 1e+00   misfit: 2.12929e+

# Results

In [19]:
inversion['results'] = inversion_results

In [20]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)